In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Local MCP Server

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [3]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [4]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0.0)

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='f94afb44-736c-4945-9f8f-cc6d5becb428'),
              AIMessage(content=[], additional_kwargs={'function_call': {'name': 'search_web', 'arguments': '{"query": "langchain-mcp-adapters library"}'}, '__gemini_function_call_thought_signatures__': {'a2ff0e8a-0ec3-45cb-bdd5-d187d34a1e7e': 'EqIDCp8DAXLI2nwna6Ysx6wmG4MnFJRXfY2JHKqRiB3wSeBGs1QFBgwFYvw/sTAU+TnrQ/8O0Z/SU3EwXqv18p9MAPxOz7QhVn95qupEPY9djHkftFO9GObUZHZdWMq3vNRFs8BEkq9tlCWXTLatPGK82hlMeu7Un9OR4zknKpO/qw2+l0DLyeoPp3Ad3T8R1FmOnnCbGg3xeDZWHsEH0ojTLAomXy2Rxsf92GWnZGBPzOuKErCskz21uMlxeWpCQIsGJG1F/hRXRRmtJPRy4+mntcxGwZCXYWiBcKCiwF/MU6EmzpGtCFax4CoE7RcFvQeMQ5hIXf3lBiQzsdhvfhMdzRW0qhv8t1y9R5XmdvMqPN2AvNkgUPr5cNalyOGY+zi9wk76617D74FyOTuuHyr3D6q/rVaG71Sc/l9nkzniWIqglLnDrTHjmW+Jy3sZ0vsUEJJ3qDDUPmAl2Rxsn0d616Yq1mT+k4SnuBjufyb+LaK7xR/nCB1reQPAOzn4+3an70ECKTMZPBnyM/BwLaHkIxauPJS3yZ6gTbU0W1kMYJ+pbA=='}}, response_me

Trace: https://smith.langchain.com/public/3adaa417-f812-4f7d-baa0-45639f889f8b/r

## Online MCP
Ref: https://mcp.so/

### Example 1: Time MCP Server

In [ ]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [9]:
agent = create_agent(
    model=model,
    tools=tools,
)

In [10]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='47de6bb2-08e0-4a35-a135-589206fbaaf5'),
              AIMessage(content=[], additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{"timezone": "America/New_York"}'}, '__gemini_function_call_thought_signatures__': {'e1731b3f-962c-407b-b7ab-50cb8532e51b': 'ErUCCrICAXLI2nzXodNAudqPrERGmbYHWwrxoMZGFuA1pOwTKiXOKczLdUAbWVA9l5BoQ8TS0C3qr0OYNUZWW8w9rkhFOQS4OYK3W5YUiXtFVK+26U/rWKvnegXQPdQ7SpToq/hmaPUdjyancvSYXkbvpbec2AL0VpTJZWpto6JNiuxOd1YvwJgdQbapbT7I6QiPbVzMn28yK4VgHLCQlrkNVBDSto9SCp16xb9747Z/OLvQo1I+9Ma6iefgPG9cvWkC6XGnvyCuhtDpyHlFdY9oA4k8E+CckspJyfbDXencL9qjP6OdQyllKVD8tysX/0qx6xEys9ABCBrBtSp+x6iC92CS10DvYDzt+BSunYuuJwI30wcQQHDh7BNFlCeHl7ll5uGRvugcIQe9ahriUsaexNvOdv+G'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bfaab-6986-77b3-8521-16f3a698e0d4-0', to

### Example 2: Kiwi.com Flight Search

In [17]:
from langchain_mcp_adapters.client import MultiServerMCPClient
client = MultiServerMCPClient(
    {"kiwi-com-flight-search": {"transport": "streamable_http", "url": "https://mcp.kiwi.com"}}
)

tools = await client.get_tools()

In [18]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0.0)

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="You are a travel agent. No follow up questions."
)

In [19]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="I want to fly from Seoul to Fukuoka on March 19th 2026")]},
    config=config
)

Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring


In [20]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='I want to fly from Seoul to Fukuoka on March 19th 2026', additional_kwargs={}, response_metadata={}, id='a0b88bdf-2acb-446b-9b0e-0a57a57c4291'),
              AIMessage(content=[], additional_kwargs={'function_call': {'name': 'search-flight', 'arguments': '{"flyFrom": "Seoul", "flyTo": "Fukuoka", "departureDate": "19/03/2026", "passengers": {"adults": 1}}'}, '__gemini_function_call_thought_signatures__': {'13132bf3-25eb-4f45-b4fc-79197695b3b7': 'EqMECqAEAXLI2nzMO16hPKss2TlH/WhVcTQmbrFFpGlesbLy18jmiY3I4fGFS3QvEWVVWVGAjoXZYKhzsKemec2Q10NA3bgdHY0FH6ObgTREtA7wVcFvJmiovjNk/V4zuFGVX+rAJcqgFfFQTP3WTUElc6ddg9vE+6YZEHj7hMqnTxY/OfjSc+V9NUPqZbhfmdkFZbCl3Ad+7y5rETag+cLOqbLqR6BL+dmOz4DG5h3UppMvmXE57/63YqGLs6SDm1rOluwCrbEEnekayDniqU3k3tPj4rnm6vHfi8TPSTd5lODAs24tjJZt4E7rRKEDbGazNEgYUyV2N2CKT/vUO+Dd2PmA/0sz4Cg23JyC46ltMz4k5vil6kTjE4rKIn2MFfPhPbQrmnyLELLKHQsuRI2zw8jNwXz2RI0IFrR6qm5iyGFPrwhVYXIvOaYxH2WfGxtV7N+YlZbPjGmwKbL45GbyZ/uhQSHeoCiXNqx37vBNGuWTCokgn+vB/Q72p+iewa

In [22]:
pprint(response["messages"][-1].content)

[{'extras': {'signature': 'ErwUCrkUAXLI2nypdAl3qq7TrZw3h1aPSfkl+cpY5BGdZH4p9hs8P6i6uvCDxhqBC28lMQRSXFfLACMjfgjKGKHm3HYP1dFGWCfpbj8k4x0rSXr3Goevk9cj8ZpfGn0zIxIkns6ta7yLGRSKcLnpzZ8wsTEwxonmDFxCbtfaoLcWW3t8kb9et1dPeJQrV3xl39/5Kin6gsZmF4XJ8q3YfvlUtOVqNttUifrCKTdlMiY5xZlR5mlos1rrpNiBwZQN4+xXsridtAsAew1x8MhVSrew6Luts/e+hMMMBEXvdpMSiy31sM5sfabHI/PHvZz6P+WSZWF+XdtoOvlD3f9L7epvoU4nC3xaIfYzDuyqXooMaEFwHAsTMId9OYBlT9KyAD+R/gCzZmzpjWgzJj14un1rHJo/6kS2um1A2rEt26u6oQXMDROBkGe80aKRUOCHz87++Nib/8ek9Xpeg5+6PSCdqq60XdA4Dr02XrGBfIzLDa74ruaB7xBcujMQHeraegVmEhc3+jiVh4vZ23sH7nD8qCivszPzO0GT6DkTVFInkLVgegO7OK2MgOYh0Kf7iWdkkvmAJhJx3sJ58OPzlamhJnmGV1wY699veTiTxn79mbSjUR3ZAvtxUqvg4nv9uloNCByuB2wmmYsgzgbFh0cSCUbBHPkZ1kgvlHRfilcbmtPZmZUZ9IjXpbzPKIE12q/PY7EkljM9CEn9T0WFs2B2IwNskywVo555+kXcc6wk0mHp3WEpWDoezp10hNt7iqQiXRDmGqrQR+HkwjU+5KtW8xFliDeK8f02Y8F+NYTQ1hSlzOk29RQeSAwTTY0ryH5duCIneDqRI5qHqvXI0vdv/wF93X6YnEc0PNAuyi6Lsu6D5atCTpCr/GCm4IA1dLGUUYFqqBMrhyLjnSZ485/aXcrmjyQUCQ/CNqVZlXJtLosxAVHH3rMnniNPx7cOEbJ40tkZYJcH7

Trace: https://smith.langchain.com/public/1c713353-6c5d-42a4-8010-b73209180bc4/r